In [59]:
import os
import re
import numpy as np
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [37]:
pwd

'/home/abzooba/Desktop/tf_2_exp/mohit_wrk'

In [38]:
path_append=os.path.join(os.curdir,'data_image')
path_append

'./data_image'

In [39]:
files=os.listdir('./data_image/')

### link for the data
- https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/numpy_bitmap?pli=1

In [41]:
# files_=[re.search('(\w.*_)(\w.*)',file).group(2) for file in files]
# for or_,new in zip(files,files_):
#     os.rename(os.path.join(path_append,or_),os.path.join(path_append,new))
    

In [42]:
files

['broom.npy',
 'alarm clock.npy',
 'cell phone.npy',
 'bee.npy',
 'baseball.npy',
 'ant.npy']

In [43]:
file_names=os.listdir('./data_image/')

In [44]:
batch_size = 128
img_rows, img_cols = 28, 28 # image dimensions

In [45]:
file_names

['broom.npy',
 'alarm clock.npy',
 'cell phone.npy',
 'bee.npy',
 'baseball.npy',
 'ant.npy']

In [47]:
num_images = 100000 ### was 100000, reduce this number if memory issues.
num_files = len(file_names) # *** we have 10 files ***
images_per_category = num_images//num_files
seed = np.random.randint(1, 10e7)
i=0
print(images_per_category)

16666


In [48]:
file_names

['broom.npy',
 'alarm clock.npy',
 'cell phone.npy',
 'bee.npy',
 'baseball.npy',
 'ant.npy']

In [50]:
data_path='./data_image/'

In [54]:
broom=np.load('./data_image/broom.npy')
broom.shape

(116927, 784)

In [55]:
i=0
for file in file_names:
    file_path = data_path + file
    print(file_path)
    x = np.load(file_path)
    x = x.astype('float32') ##normalize images
    x /= 255.0
    print(len(x))
    y = [i] * len(x) # create numeric label for this image
    x = x[:images_per_category] # get the sample of images
    y = y[:images_per_category] # get the sample of labels
    if i == 0:
        x_all =x
        y_all =y
    else:
        x_all =np.concatenate((x,x_all),axis=0)
        y_all =np.concatenate((y,y_all),axis=0)
    i += 1

./data_image/broom.npy
116927
./data_image/alarm clock.npy
123399
./data_image/cell phone.npy
121130
./data_image/bee.npy
120890
./data_image/baseball.npy
135375
./data_image/ant.npy
124612


In [64]:

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.3, random_state=2)


In [65]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((69997, 784), (29999, 784), (69997,), (29999,))

In [66]:
x_train=x_train.reshape(-1,img_rows,img_cols,1)
x_train.shape

(69997, 28, 28, 1)

In [67]:
x_test=x_test.reshape(-1,img_rows,img_cols,1)
x_test.shape

(29999, 28, 28, 1)

In [68]:
input_shape=x_train.shape[1:]
input_shape

(28, 28, 1)

In [69]:
num_files

6

In [70]:
y_train=tf.keras.utils.to_categorical(y_train,num_classes=num_files)
y_train.shape

(69997, 6)

In [72]:
y_test=tf.keras.utils.to_categorical(y_test,num_classes=num_files)
y_test.shape

(29999, 6)

In [73]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1,random_state=2)
x_train.shape

(62997, 28, 28, 1)

In [81]:
model=tf.keras.Sequential()

In [82]:
model.add(layer=tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))

In [83]:
model.add(layer=tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(layer=tf.keras.layers.BatchNormalization())
model.add(layer=tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(layer=tf.keras.layers.BatchNormalization())

model.add(layer=tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3), padding='same',activation='relu'))          
model.add(layer=tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(layer=tf.keras.layers.BatchNormalization())

model.add(layer=tf.keras.layers.Flatten())
model.add(layer=tf.keras.layers.Dense(units=128,activation='relu'))
model.add(layer=tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(units=num_files,activation='sigmoid'))          

In [85]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [90]:
epochs=20

In [108]:
callbacks=[tf.keras.callbacks.TensorBoard(log_dir='./logs/')]
model.fit(x=x_train,y=y_train,batch_size=batch_size,callbacks=callbacks,verbose=1,validation_data=(x_valid,y_valid),epochs=epochs)

In [92]:
model.evaluate(x=x_test,y=y_test)

29999/29999 [==============================] - 3s 101us/sample - loss: 0.2041 - accuracy: 0.9352


[0.20413832364926685, 0.93519783]

In [94]:
[os.path.splitext(file)[0] for file in file_names]

['broom', 'alarm clock', 'cell phone', 'bee', 'baseball', 'ant']

In [104]:
import os
labels = [os.path.splitext(file)[0] for file in file_names]

print(labels)
print("\nFor each pair in the following, the first label is predicted, second is actual\n")

['broom', 'alarm clock', 'cell phone', 'bee', 'baseball', 'ant']

For each pair in the following, the first label is predicted, second is actual



In [105]:
for i in range(20):
    t = np.random.randint(len(x_test) )
    x1= x_test[t]
    x1 = x1.reshape(1,28,28,1)
    p = model.predict(x1)
    print("-------------------------")
    print(labels[np.argmax(p)])
    print(labels[np.argmax(y_test[t])])

-------------------------
ant
ant
-------------------------
alarm clock
alarm clock
-------------------------
ant
ant
-------------------------
cell phone
cell phone
-------------------------
broom
broom
-------------------------
cell phone
cell phone
-------------------------
cell phone
cell phone
-------------------------
baseball
baseball
-------------------------
alarm clock
alarm clock
-------------------------
broom
broom
-------------------------
cell phone
cell phone
-------------------------
alarm clock
alarm clock
-------------------------
bee
bee
-------------------------
cell phone
cell phone
-------------------------
cell phone
cell phone
-------------------------
alarm clock
alarm clock
-------------------------
cell phone
cell phone
-------------------------
cell phone
cell phone
-------------------------
alarm clock
alarm clock
-------------------------
alarm clock
alarm clock


In [106]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 3, 3, 64)         

In [110]:
os.mkdir('./h5_files')

In [111]:
import h5py
with h5py.File('./h5_files/x_train.h5', 'w') as hf:
    hf.create_dataset("QuickDraw", data=x_train)
with h5py.File('./h5_files/y_train.h5', 'w') as hf:
    hf.create_dataset("QuickDraw", data=y_train)
with h5py.File('./h5_files/x_test.h5', 'w') as hf:
    hf.create_dataset("QuickDraw", data=x_test)
with h5py.File('./h5_files/y_test.h5', 'w') as hf:
    hf.create_dataset("QuickDraw", data=y_test)

In [125]:
from os import walk
import os
data_path = "./data_image/" # folder for image files

for (dirpath, dirnames, filenames) in walk(data_path):
    pass # filenames accumulate in list 'filenames'

labels = [os.path.splitext(file)[0] for file in filenames]
print(labels)

['broom', 'alarm clock', 'cell phone', 'bee', 'baseball', 'ant']


### predictions with CPU

In [132]:
with tf.device('/cpu:0'):
    for i in range(10):
        t = np.random.randint(len(x_test) )
        x1= x_test[t]
        x1 = x1.reshape(1,28,28,1)
        p = model.predict(x1)
        y1 = y_test[t]
        print("-------------------------")
        print(labels[np.argmax([p])])
        print(labels[np.argmax(y1)])
        print("-------------------------")

-------------------------
alarm clock
alarm clock
-------------------------
-------------------------
bee
bee
-------------------------
-------------------------
alarm clock
alarm clock
-------------------------
-------------------------
ant
ant
-------------------------
-------------------------
alarm clock
alarm clock
-------------------------
-------------------------
ant
ant
-------------------------
-------------------------
bee
bee
-------------------------
-------------------------
broom
broom
-------------------------
-------------------------
cell phone
cell phone
-------------------------
-------------------------
ant
ant
-------------------------
